ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [3]:
PINECONE_API_KEY = "pcsk_5PfHxm_KaWUt49aUh8jHKToWzmCkMNeehdCkuf6aGihwfZLR4ZH5BFGLBjMSnCvCk4KAV1"
PINECONE_API_ENV = "gcp-starter"


In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))


length of my chunk: 5859


In [8]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
# Use new Pinecone client syntax
from pinecone import Pinecone as PineconeClient
pc = PineconeClient(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chatbot")

# Store embeddings using LangChain's Pinecone vectorstore wrapper
docsearch = Pinecone(index, embeddings.embed_query, "text")
texts = [t.page_content for t in text_chunks]
metadatas = [t.metadata for t in text_chunks]
docsearch.add_texts(texts, metadatas=metadatas)
print("Embeddings stored in Pinecone index using new client.")

Upserted vectors: 100%|██████████| 5859/5859 [01:09<00:00, 83.78it/s]

Embeddings stored in Pinecone index using new client.


In [13]:
# Patch for Pinecone/LC API breaking change: monkey-patch Pinecone index.query to force keyword arguments
from functools import wraps
def patch_pinecone_query(index):
    orig_query = index.query
    @wraps(orig_query)
    def new_query(*args, **kwargs):
        if args:
            # Assume first arg is vector, second is top_k, third is namespace
            if len(args) == 1:
                kwargs['vector'] = args[0]
            elif len(args) == 2:
                kwargs['vector'] = args[0]
                kwargs['top_k'] = args[1]
            elif len(args) == 3:
                kwargs['vector'] = args[0]
                kwargs['top_k'] = args[1]
                kwargs['namespace'] = args[2]
            args = ()
        return orig_query(*args, **kwargs)
    index.query = new_query
    return index

from pinecone import Pinecone as PineconeClient
pc = PineconeClient(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chatbot")
patch_pinecone_query(index)

# Use LangChain's Pinecone vectorstore wrapper to connect to the index
docsearch = Pinecone(index, embeddings.embed_query, "text")

query = "What are Allergies"
docs = docsearch.similarity_search(query=query, k=3)
print("Result", docs)

Result [Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of', metadata={'page': 135.0, 'source': 'data\\Medical_book (1).pdf'}), Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection

In [14]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [16]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [17]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])